<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       RegressionEvaluator function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>RegressionEvaluator function computes metrics to evaluate and compare multiple models and summarizes how close predictions are to their expected values. It takes the actual and predicted values of the dependent variables to calculate specified metrics, you choose which metrics you want to calculate from a list of supported metrics.
         <br> In this notebook we will see how we can use the RegressionEvaluator function available in Vantage.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import *

# Modify the following to match the specific client environment settings
display.max_rows = 5
configure.val_install_location = 'val'

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_RegressionEvaluator_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_TitanicSurvival_local');"        # Takes 30 seconds
#%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_TitanicSurvival_cloud');" 

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
tdf = DataFrame(in_schema("DEMO_TitanicSurvival", "Passenger_Data"))
print("Shape of the data: ", tdf.shape)
tdf

<p style = 'font-size:16px;font-family:Arial'>Let us create Regression model on this data.</p>

In [ ]:
# First generate linear regression model using LinReg() function from 'valib'.
lin_reg_obj = valib.LinReg(data=tdf,
                           columns=["age", "survived", "pclass"],
                           response_column="fare")

In [ ]:
# Score the data using the linear regression model generated above.
obj = valib.LinRegPredict(data=tdf,
                          model=lin_reg_obj.model,
                          accumulate = "fare",
                          response_column="fare_prediction")

<p style = 'font-size:16px;font-family:Arial'>Now that our model is created and we have predictions from the model. We can use RegressionEvaluator function to get the model metrics.Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(RegressionEvaluator)

In [ ]:
# Compute 'RMSE', 'R2' and 'FSTAT' metrics to evaluate
#             the model.
RegressionEvaluator_out = RegressionEvaluator(data = obj.result,
                                              observation_column = "fare",
                                              prediction_column = "fare_prediction",
                                              freedom_degrees = [1, 2],
                                              independent_features_num = 2,
                                              metrics = ['RMSE','R2','FSTAT'])

# Print the result DataFrame.
RegressionEvaluator_out.result

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. Cleanup</b>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_TitanicSurvival');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>RegressionEvaluator function reference: <a href = 'https://docs.teradata.com/search/all?query=RegressionEvaluator&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>